<a href="https://colab.research.google.com/github/cantbelieveimshook/Autoencoder-transfer-learning/blob/main/Autoencoder_benchmarking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# connects colab to your google drive
# skip if your dataset is not on google drive or you're not using colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import keras
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, BatchNormalization, Conv2DTranspose
from tensorflow.keras.utils import  plot_model
from keras.preprocessing.image import ImageDataGenerator
import h5py
import io
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import pickle
from scipy.stats import norm
from keras.constraints import UnitNorm, Constraint

In [ ]:
# filepath = '/content/drive/MyDrive/Big Data REU 2022 Team 1/REU 2022 Research/Datasets/newly_organized_pngs/'
filepath = '/content/drive/MyDrive/Big Data REU 2022 Team 1/REU 2022 Research/Datasets/new-test-data_Seraj/fft-denoised/'
os.chdir(filepath) # changes the current working directory to the file path specified. This directory should be the directory of data you plan on using for the model

In [ ]:
batch_size = 32 # set batch_size to 32 because it seems to provide the best results given our dataset

# this ImageDataGenerator is used for training, validation, and testing generators. No preprocessing besides dividing all values by 255 is carried out
train_datagen = ImageDataGenerator(
        rescale=1./255
        )

train = train_datagen.flow_from_directory(
        filepath + "train/",
        target_size=(256, 256), # images, regardless of their original size, are shrunk down to 256 x 256 x 3
        batch_size=batch_size,
        color_mode = 'rgb',
        class_mode='binary', # for binary classification
        shuffle=False # do not shuffle images
    )

val = train_datagen.flow_from_directory(
        filepath + "validation/",
        target_size=(256, 256),
        batch_size=batch_size,
        color_mode = 'rgb',
        class_mode='binary',
        shuffle=False
    )

test = train_datagen.flow_from_directory(
        filepath + "test/",
        target_size=(256, 256),
        batch_size=batch_size,
        color_mode = 'rgb',
        class_mode='binary',
        shuffle=False
    )


Found 710 images belonging to 2 classes.
Found 140 images belonging to 2 classes.
Found 236 images belonging to 2 classes.


In [ ]:
# this model has the exact same architecture and regularizers as the classification model, but without any pre-trained weights
# if images are not rgb, input_shape should be changed to (256, 256, 1) instead of (256, 256, 3)
baseline = models.Sequential()
baseline.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same', input_shape = (256, 256, 3), kernel_regularizer = keras.regularizers.l2(l = 0.01)))
baseline.add(layers.MaxPooling2D((2, 2), padding='same'))
baseline.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same', kernel_regularizer = keras.regularizers.l2(l = 0.01)))
baseline.add(layers.MaxPooling2D((2, 2), padding='same'))
baseline.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
baseline.add(layers.MaxPooling2D((2, 2), padding='same'))
baseline.add(keras.layers.Flatten())
baseline.add(keras.layers.Dense(64, activation='relu'))
baseline.add(keras.layers.Dropout(0.3))
baseline.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
baseline.compile(
        optimizer='Adam',
        metrics=['accuracy'],
        loss='binary_crossentropy')

In [ ]:
epochs = 100

h = baseline.fit(
    train,
    epochs=epochs,
    validation_data=val,
    verbose=1)

Epoch 1/100
23/23 [==============================] - 345s 14s/step - loss: 0.8639 - accuracy: 0.4324 - val_loss: 0.8346 - val_accuracy: 0.4929
Epoch 2/100
23/23 [==============================] - 17s 739ms/step - loss: 0.8258 - accuracy: 0.4930 - val_loss: 0.8167 - val_accuracy: 0.5000
Epoch 3/100
23/23 [==============================] - 19s 821ms/step - loss: 0.8095 - accuracy: 0.5000 - val_loss: 0.8021 - val_accuracy: 0.5000
Epoch 4/100
23/23 [==============================] - 17s 737ms/step - loss: 0.7965 - accuracy: 0.5000 - val_loss: 0.7905 - val_accuracy: 0.5000
Epoch 5/100
23/23 [==============================] - 17s 732ms/step - loss: 0.7861 - accuracy: 0.5000 - val_loss: 0.7813 - val_accuracy: 0.5000
Epoch 6/100
23/23 [==============================] - 17s 760ms/step - loss: 0.7779 - accuracy: 0.5000 - val_loss: 0.7739 - val_accuracy: 0.5000
Epoch 7/100
23/23 [==============================] - 17s 727ms/step - loss: 0.7711 - accuracy: 0.5000 - val_loss: 0.7677 - val_accuracy: 

In [ ]:
print("Average training accuracy: ",sum(h.history['accuracy'])/100)
print("Average training loss: ",sum(h.history['loss'])/100)
print("Average validation accuracy: ",sum(h.history['val_accuracy'])/100)
print("Average validation loss: ",sum(h.history['val_loss'])/100)

Average training accuracy:  0.4993999996781349
Average training loss:  1.6892051362991334
Average validation accuracy:  0.4993333330750465
Average validation loss:  0.8969045370817185


In [ ]:
plt.plot(h.history['accuracy'])
plt.plot(h.history['val_accuracy'])
plt.title('Training and Validation Accuracies')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc = 'upper left')
plt.show()

In [ ]:
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.title('Training and Validation Losses')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc = 'upper left')
plt.show()

In [ ]:
_, acc = baseline.evaluate(test, verbose = 0)
print('> %.3f' % (acc * 100))

> 50.000


In [ ]:
import sklearn.metrics as metrics


pred = baseline.predict(val)

print("Confusion Matrix: \n")
true_classes = val.classes
class_labels = list(val.class_indices.keys())
pred = np.round(pred)
confusion_matrix = metrics.confusion_matrix(y_true=true_classes, y_pred=pred)
confusion_matrix


Confusion Matrix: 



array([[ 0, 30],
       [ 0, 30]])